## Load the dataset

In [ ]:
import numpy as np

# Load the dataset
data = np.load('your_dataset.npz')

# Inspect the keys (names of stored arrays)
print(data.files)


## Extract Arrays

In [ ]:
# Extract arrays
X_train = data['X_train']
Y_train = data['Y_train']
X_val = data['X_val']
Y_val = data['Y_val']
X_test = data['X_test']
Y_test = data['Y_test']


## Data Preprocessing

In [ ]:
X_train = X_train / 1.0  # Scale values if needed (e.g., divide by max value)
X_val = X_val / 1.0
X_test = X_test / 1.0


In [ ]:
from tensorflow.keras.utils import to_categorical

Y_train = to_categorical(Y_train, num_classes=7)  # Assuming 7 possible moves
Y_val = to_categorical(Y_val, num_classes=7)
Y_test = to_categorical(Y_test, num_classes=7)


## Check Shapes

In [ ]:
print(X_train.shape, Y_train.shape)  # E.g., (num_samples, 6, 7, 2) and (num_samples, 7)


## CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(6, 7, 2)),  # Adjust shape if needed
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # Output layer with 7 classes
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
cnn_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=20, batch_size=32)

## Transformer

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf

def transformer_block(inputs, num_heads, key_dim, ff_dim):
    # Multi-head attention
    attn_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(inputs, inputs)
    attn_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attn_output)

    # Feedforward network
    ffn = tf.keras.Sequential([
        Dense(ff_dim, activation='relu'),
        Dense(inputs.shape[-1])
    ])
    ffn_output = ffn(attn_output)
    ffn_output = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn_output + ffn_output)

    return ffn_output

# Build Transformer Model
input_layer = Input(shape=(6, 7, 2))  # Adjust shape as needed
flattened = Flatten()(input_layer)  # Flatten 6x7x2 to 84
transformer_output = transformer_block(flattened, num_heads=4, key_dim=8, ff_dim=32)
output_layer = Dense(7, activation='softmax')(transformer_output)

transformer_model = Model(inputs=input_layer, outputs=output_layer)
transformer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
transformer_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=20, batch_size=32)

## Test

In [ ]:
cnn_accuracy = cnn_model.evaluate(X_test, Y_test)
transformer_accuracy = transformer_model.evaluate(X_test, Y_test)
print(f"CNN Accuracy: {cnn_accuracy[1]}")
print(f"Transformer Accuracy: {transformer_accuracy[1]}")
